In [1]:
!pip install galsim

import GalSimData as GSD
import CMDN
import TrainCMDNnewGPU as TCMDN

import numpy as np
import math
import pickle
import torch
import seaborn as sns
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import rbf_kernel
from scipy.spatial.distance import pdist
from scipy.stats import t
from scipy.stats import norm
import matplotlib.pyplot as plt
import time
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 20.1 MB/s eta 0:00:00


In [2]:
import random

def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

In [3]:
def h_mat (X_data, Y_data):

    gamma = np.median(np.abs(X_data - Y_data))**(-2)
    kernel_XX = rbf_kernel(X_data,X_data, gamma = gamma)
    kernel_YY = rbf_kernel(Y_data,Y_data, gamma = gamma)
    kernel_XY = rbf_kernel(X_data,Y_data, gamma = gamma)
    kernel_YX = rbf_kernel(Y_data,X_data, gamma = gamma)

    return kernel_XX + kernel_YY - kernel_XY - kernel_YX

def EMMD (knn_matrix, h_mat, n_data, k_n):

    h_mat_knn = np.multiply(h_mat, knn_matrix)

    emmd_val = np.sum(h_mat_knn)/np.sqrt(n_data * k_n)

    return (emmd_val)

def samp_var (knn_matrix, h_mat, n_data, k_n):

    double_edge = ((knn_matrix + knn_matrix.T) == 2).astype(int)
    h2_mat = np.square(h_mat)
    h2_mat_term1 = np.multiply(h2_mat, knn_matrix)
    h2_mat_term2 = np.multiply(h2_mat, double_edge)
    S_squared = (np.sum(h2_mat_term1) + np.sum(h2_mat_term2))/(n_data * k_n)

    return np.sqrt(S_squared)

def emmd_test_stat (knn_matrix, X_data, Y_data, n_data, k_n):

    H_mat = h_mat(X_data, Y_data)
    emmd_val = EMMD(knn_matrix, H_mat, n_data, k_n)
    S_val = samp_var(knn_matrix, H_mat, n_data, k_n)
    return np.absolute(emmd_val/S_val)

In [4]:
def sample_alpha_lambda(sample_size):

    alpha_sample = np.random.uniform(-math.pi/2,math.pi/2, size=sample_size)
    lambda_sample = np.random.uniform(-0.1,0.1, size=sample_size) + 0.75

    return alpha_sample, lambda_sample

def sample(pi, mu, sigma):
        mixture = torch.normal(mu, sigma)
        k = torch.multinomial(pi, 1, replacement=True).squeeze()
        result = mixture[range(k.size(0)), k]
        return result

In [5]:
def z_sample (alpha_sample, lambda_sample,\
              mu1, mu2_1, mu2_2, sigma1 = 1, sigma2 = 0.25, alpha_cutoff = 0):
  n_size = len(alpha_sample)
  U_ind = np.random.binomial(n=1, p=0.5, size=n_size)
  V_ind = np.random.binomial(n=1, p=0.5, size=n_size)
  z_ind = np.random.binomial(n=1, p=0.5, size=n_size)
  z = np.zeros(n_size)

  for i in range(n_size):
    if alpha_sample[i] > alpha_cutoff:
      U = np.random.normal(mu1,sigma1,1)
      V = np.random.normal(-mu1,sigma1,1)
    else:
      U = U_ind[i] * np.random.normal(mu2_1, sigma2, 1) + (1-U_ind[i]) * np.random.normal(mu2_2, sigma2, 1)
      V = V_ind[i] * np.random.normal(-mu2_1, sigma2, 1) + (1-V_ind[i]) * np.random.normal(-mu2_2, sigma2, 1)

    z[i] = z_ind[i] * U + (1-z_ind[i]) * V

  return z

In [6]:
def train_model (n_train, mu1, mu2_1, mu2_2, alpha_cutoff,\
                 k = 2, downsampling = 20):

  alpha_sample,lambda_sample = sample_alpha_lambda(n_train)
  GSD.main(alpha_sample, lambda_sample, downsampling = downsampling)

  with open("data.pkl", 'rb') as handle:
    data = pickle.load(handle)

  z_train = z_sample(alpha_sample, lambda_sample, mu1 = mu1, mu2_1 = mu2_1,\
                     mu2_2 = mu2_2, alpha_cutoff = alpha_cutoff)

  trained_model = TCMDN.main(data, z_train, n_train = n_train, k = k,\
                             width = 20, height = 20, epochs = 5000)

  return (trained_model)

In [7]:
def test_out (trained_model, n_test, k_knn,\
              mu1, mu2_1, mu2_2, sigma1 = 1, sigma2 = 0.25, alpha_cutoff = 0,\
              indicator_image = 0):

  alpha_sample,lambda_sample = sample_alpha_lambda(n_test)
  GSD.main(alpha_sample, lambda_sample, downsampling = 20)

  with open("data.pkl", 'rb') as handle:
      data = pickle.load(handle)

  out_test = TCMDN.test_model(trained_model, data, n_test)

  Samples_test = sample(out_test[0], out_test[1], out_test[2])

  galaxy_test = data['galaxies_generated']

  gal2 = galaxy_test.reshape(n_test,-1)

  gal_knn = kneighbors_graph(gal2, k_knn, include_self = False).toarray()

  z_test = z_sample(alpha_sample, lambda_sample, mu1, mu2_1, mu2_2,\
                    alpha_cutoff = alpha_cutoff)
  X = z_test.reshape(-1,1)
  Y = np.array(Samples_test.cpu().detach().numpy().tolist()).reshape(-1,1)

  if indicator_image == 0:

    bins = np.linspace(-5, 5, 30)
    plt.hist(X, bins, alpha=0.5, density = True)
    plt.hist(Y, bins, alpha=0.5, density = True)
    plt.show()

  return emmd_test_stat(gal_knn, X, Y, n_test, k_knn)>norm.ppf(0.975)

def test_power (trained_model, n_test, k_knn,\
                mu1, mu2_1, mu2_2, sigma1 = 1, sigma2 = 0.25, alpha_cutoff = 0,\
                n_rep = 100):

  test_out_vec = np.zeros(n_rep)
  for i in range(n_rep):
    test_out_vec[i] = test_out(trained_model, n_test = n_test, k_knn = k_knn,\
                               mu1 = mu1, mu2_1 = mu2_1, mu2_2 = mu2_2,\
                               alpha_cutoff = alpha_cutoff,\
                               indicator_image = i)
    print (i)

  return (np.sum(test_out_vec)/n_rep)


In [8]:
from IPython.display import display, clear_output

n_train = 20000
n_test1 = 1000; n_test2 = 2500
mu1 = 2; mu2_1 = 3; mu2_2 = 0.5
alpha_cutoff = [-math.pi/3, -math.pi/4, -math.pi/6, -math.pi/12, \
                0, math.pi/12, math.pi/6, math.pi/4, math.pi/3]

# alpha_cutoff = [-math.pi/6]

alpha_power11 = []; alpha_power12 = []; alpha_power21 = []
alpha_power22 = []

n_rep = 100
k_knn1 = 50; k_knn2 = 100

for i in range(len(alpha_cutoff)):

  set_all_seeds(1)

  trained_model = train_model(n_train = n_train, mu1 = mu1, mu2_1 = mu2_1,\
                              mu2_2 = mu2_2, alpha_cutoff = alpha_cutoff[i],\
                              k = 2)

  alpha_test_power11 = test_power(trained_model = trained_model, n_test = n_test1,\
                                k_knn = k_knn1, mu1 = mu1, mu2_1 = mu2_1,\
                                mu2_2 = mu2_2, alpha_cutoff = alpha_cutoff[i],\
                                n_rep = n_rep)
  print (alpha_test_power11)
  alpha_test_power12 = test_power(trained_model = trained_model, n_test = n_test1,\
                                k_knn = k_knn2, mu1 = mu1, mu2_1 = mu2_1,\
                                mu2_2 = mu2_2, alpha_cutoff = alpha_cutoff[i],\
                                n_rep = n_rep)
  print (alpha_test_power12)
  alpha_test_power21 = test_power(trained_model = trained_model, n_test = n_test2,\
                                k_knn = k_knn1, mu1 = mu1, mu2_1 = mu2_1,\
                                mu2_2 = mu2_2, alpha_cutoff = alpha_cutoff[i],\
                                n_rep = n_rep)
  print (alpha_test_power21)
  # alpha_test_power22 = test_power(trained_model = trained_model, n_test = n_test2,\
  #                               k_knn = k_knn2, mu1 = mu1, mu2_1 = mu2_1,\
  #                               mu2_2 = mu2_2, alpha_cutoff = alpha_cutoff[i],\
  #                               n_rep = n_rep)
  alpha_power11.append(alpha_test_power11)
  alpha_power12.append(alpha_test_power12)
  alpha_power21.append(alpha_test_power21)
  # alpha_power22.append(alpha_test_power22)
  clear_output(wait=True)
  print (alpha_test_power11, alpha_test_power12, alpha_test_power21)

alpha_df11 = pd.DataFrame(alpha_power11)
alpha_df12 = pd.DataFrame(alpha_power12)
alpha_df21 = pd.DataFrame(alpha_power21)
# alpha_df22 = pd.DataFrame(alpha_power22)
alpha_df11.to_csv('powermu2205_n1000_k50.csv', index=False)
alpha_df12.to_csv('powermu2205_n1000_k100.csv', index=False)
alpha_df21.to_csv('powermu2205_n2500_k50.csv', index=False)
# alpha_df22.to_csv('powermu220_n2500_k100.csv', index=False)

0.77 0.88 1.0


In [9]:
from google.colab import files
files.download('powermu2205_n1000_k50.csv')
files.download('powermu2205_n1000_k100.csv')
files.download('powermu2205_n2500_k50.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>